In [39]:
import pytest
import random
import math
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
%matplotlib inline
import time
from sklearn.cluster import KMeans

In [18]:
def GenerateData(select, CL1=7, CL2=14, CL3=21, seed=None):
    """Generate data for the ideal case (clusters well separated). 
        This function can genearate 2 or 3 clusters.    

        Args:
            select: Which dataset to generate, can only input 1 or 2.
            CL1: First cluster's size.
            CL2: Second cluster's size.
            CL3: Third cluster's size.
            seed: Set seed for the random number generators.

        Returns:
            A numpy array for the generated dataset.

    """
    
    # set seed if user desired
    if seed:
        random.seed(100)
    # choose to generate two datasets
    if select == 1:
        data = np.zeros((CL2,2))
        for i in range(0,CL1):
            data[i,0] = 1 + random.random()
            data[i,1] = 1 + random.random()
        
        for i in range(CL1,CL2):
            data[i,0] = 3 + random.random()
            data[i,1] = 3 + random.random()
    # choose to generate three datasets
    elif select == 2:
        data = np.zeros((CL3,2))
        for i in range(0,CL1):
            data[i,0] = 1 + random.random()
            data[i,1] = 1 + random.random()
        
        for i in range(CL1,CL2):
            data[i,0] = 3 + random.random()
            data[i,1] = 1 + random.random()

        for i in range(CL2,CL3):
            data[i,0] = 1.5 + random.random()
            data[i,1] = 3 + random.random()
    return data

def CreateCircle(n, seed = None):
    """Generate data for the case that do not correspond to non-convex region. 
        This function can genearate points that form circles.    

        Args:
            n: How many circles the user wants.

        Returns:
            A numpy array for the generated dataset.

    """
    if seed:
        np.random.seed(100)
    # generate a center for circles
    u=n*np.random.random()
    # generate an array of radius
    d= np.linspace(1,5*n,n)
    # generate an array of positions where the circles cross the x axis
    r=u+d
    
    df=np.zeros((n*200,2))
    for i in range(n):
        # an array of positions on x axis
        x=np.linspace(-r[i],r[i],100)
        # corresponding y axis > 0for the circle
        y=np.sqrt(r[i]**2-x**2)
        # corresponding y axis < 0for the circle
        y2=-np.sqrt(r[i]**2-x**2)
        for j in range(2*i*100,(2*i+1)*100):
            df[j,0]=x[j-2*i*100]
            df[j,1]=y[j-2*i*100]
        for j in range((2*i+1)*100,(2*i+2)*100):
            df[j,0]=x[j-(2*i+1)*100]
            df[j,1]=y2[j-(2*i+1)*100]
    
    # Add noise
    df = df + np.random.random((n*200,2))
    
    return df

def CalculateAffinity(data, sigma = 0.5):
    """Create an distance matrix with the given data. This is Step 1 in the Spectral Clustering Algorithm.
    Step 1: Affinity matrix A defined by A_ij=exp(-||s_i-s_j||^2/2*sigma^2), and A_ii=0.

        Args:
            data: Data to be used to calculate Affinity.
            sigma: Standard deviation that can be chosen by user.

        Returns:
            A numpy array for the calculated affinity matrix.

    """
    #find number of columns in the dataset
    col_size = data.shape[0]
    #find number of rows in the dataset
    dim = data.shape[1]
    #create an empty numpy array of correct size
    affinity = np.zeros((col_size,col_size))

    for i in range(col_size):
        for j in range(col_size):
            dist_tmp = 0
            for k in range(dim):
                # calculate the euclidiean distance
                dist_tmp +=(data[i,k]-data[j,k])**2
            dist = math.sqrt(dist_tmp)
            # calculate the affinity matrix A_ij=exp(-||s_i-s_j||^2/2*sigma^2)
            affinity[i,j] = math.exp(-dist/(2*sigma**2))
    # A_ii=0
    A = (np.ones(affinity.shape) - np.eye(affinity.shape[0]))*affinity
    
    return A

def Spectral(k, A):
    """Create cluster labels for points from k-means using the normalized matrix Algorithm Step 2-6. 

    Step 2: Define D to be the diagonal matrix whose (i,i)-element is the sum of A's i_th
row,and construct the matrix L =D^(-l/2)AD^(-l/2).

    Step 3: Find Xl , X2 , ... , Xk, the k largest eigenvectors of L (chosen to be orthogonal to each other in the case of repeated eigenvalues), and form the matrix X = [XIX2 . . . Xk) E R^(nxk) by stacking the eigenvectors in columns.

    Step 4: Form the matrix Y from X by renormalizing each of X's rows to have unit length (i.e. Y_ij = X_ij/sum(X_ij^2)^(1/2).

    Step 5: Treating each row of Y as a point in R^k , cluster them into k clusters via K-means or any other algorithm.

    Step 6: Finally, assign the original point s_i to cluster j if and only if row i of the matrix Y was assigned to cluster j.


        Args:
            k: The number of clusters desired.
            A: The affinity matrix.

        Returns:
            A numpy array of the cluster assignment for each point.

    """
    # compute the degree matrix
    
    D = np.diag(np.sum(A, axis=0))


    #compute the normalized laplacian matrix
    NL1 = np.zeros((A.shape))
    for i in range(A.shape[0]):
         for j in range(A.shape[1]):
                NL1[i,j] = A[i,j]/(math.sqrt(D[i,i])*math.sqrt(D[j,j]))

    eigvalue,eigvector = la.eig(NL1)
    sort_index = np.argsort(eigvalue)
    X = eigvector[:,sort_index[-k:][::-1]]

    
    
    #construct the normalized matrix U from the obtained eigen vectors
    Y = np.zeros((X.shape))
    for i in range(X.shape[0]):
        n = math.sqrt(sum(X[i,:]**2));   
        Y[i,:] = X[i,:]/n 

    #Apply k-means
    clust_result = KMeans(n_clusters=k, random_state=10)
    cluster_labels = clust_result.fit_predict(Y)
    indicator=np.loadtxt(cluster_labels,dtype='int32')
    ID=indicator.tolist()
    IDX=np.asarray(ID) 
    return IDX

In [7]:
withSeed0 = np.array([ 1.14566926,  1.454927  ])

In [8]:
"""Generate Data"""
#General Case
data1 = GenerateData(2,seed=True)
assert data1.shape == (21,2)
assert np.allclose(data1[0], withSeed0)
assert type(data1) == np.ndarray
#Edge Case
with pytest.raises(TypeError):
        GenerateData()
with pytest.raises(UnboundLocalError):
        GenerateData(0)

In [21]:
withSeedCircle0 = np.array([-1.8084405 ,  0.42451759])

In [34]:
"""CreateCircle"""
#General Case
data2 = CreateCircle(2,seed=True)
assert data2.shape == (400,2)
assert np.allclose(data2[0], withSeedCircle0)
assert type(data2) == np.ndarray
#Edge Case
with pytest.raises(TypeError):
        CreateCircle()
assert type(CreateCircle(0)) == np.ndarray


In [36]:
withSeedAffinity0 = np.array([ 0.        ,  0.26003482,  0.30932548,  0.26768046,  0.8771766 ,
         0.37686546,  0.19778772,  0.01261911,  0.01571137,  0.01671333,
         0.00631755,  0.01426118,  0.0074362 ,  0.00462141,  0.02035262,
         0.01930505,  0.00833584,  0.00747444,  0.01259086,  0.00954783,
         0.01525707])

In [37]:
"""Calculate Affinity"""
#General case
affinity = CalculateAffinity(data1)
assert affinity.shape == (21,21)
assert np.allclose(affinity[0],withSeedAffinity0)
assert type(affinity) == np.ndarray
#Edge Case
with pytest.raises(AttributeError):
        CalculateAffinity(0)
with pytest.raises(TypeError):
        CalculateAffinity()

In [42]:
withSeedSpectral = np.array([2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1])

In [45]:
"""Spectral"""
#General case
spectral = Spectral(3, affinity)
assert spectral.shape == (21,)
assert np.allclose(spectral,withSeedSpectral)
assert type(affinity) == np.ndarray
#Edge case
with pytest.raises(TypeError):
        Spectral()
with pytest.raises(TypeError):
        Spectral(0)
with pytest.raises(ValueError):
        Spectral(0,0)
